# Описание проекта

Нужно решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# 1. Загрузка и подготовка данных

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
from scipy import stats as st

In [2]:
data_0 = pd.read_csv('geo_data_0.csv')
data_1 = pd.read_csv('geo_data_1.csv')
data_2 = pd.read_csv('geo_data_2.csv')

In [3]:
data_0.info()
data_0.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
data_1.info()
data_1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


Проверим данные на дубликаты.

In [5]:
print('Количество дуликатов =', data_0.duplicated().sum())
print('Количество дуликатов =', data_1.duplicated().sum())
print('Количество дуликатов =', data_2.duplicated().sum())

Количество дуликатов = 0
Количество дуликатов = 0
Количество дуликатов = 0


Полных дубликатов нет. Проверим дубликаты в id месторождения. 

In [6]:
print('Количество дубликатов id =', data_0['id'].duplicated().sum())
print('Количество дубликатов id =', data_1['id'].duplicated().sum())
print('Количество дубликатов id =', data_2['id'].duplicated().sum())

Количество дубликатов id = 10
Количество дубликатов id = 4
Количество дубликатов id = 4


Так как у месторождени с одинаковым id разные признаки, можем предположить, что у этих месторождений пробы брались несколько раз.
Так как столбец id не влияет на принятие решения при обучении, мы можем не удалять эти дубликаты.

# 2. Обучение и проверка модели

Выделим целевой признак.
Признаки - это столбцы f0, f1 и f2
Целевой признак - это столбец product

Обучим модель для первого месторождения data_0

In [7]:
features_0 = data_0.drop(['id', 'product'], axis=1)
target_0 = data_0['product']
features_train_0, features_valid_0, target_train_0, target_valid_0 = (train_test_split(features_0, 
                                                                               target_0, 
                                                                               test_size = 0.25, 
                                                                               random_state=42))

In [8]:
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
predictions_0 = model_0.predict(features_valid_0)
rmse_0 = (mean_squared_error(target_valid_0, predictions_0))**0.5
ave_0 = target_0.mean()

print('RMSE модели =', rmse_0)
print('Средний запас сырья =', ave_0)

RMSE модели = 37.75660035026169
Средний запас сырья = 92.50000000000001


Обучим модель для второго месторождения data_1

In [9]:
features_1 = data_1.drop(['id', 'product'], axis=1)
target_1 = data_1['product']
features_train_1, features_valid_1, target_train_1, target_valid_1 = (train_test_split(features_1, 
                                                                               target_1, 
                                                                               test_size = 0.25, 
                                                                               random_state=42))

In [10]:
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predictions_1 = model_1.predict(features_valid_1)
rmse_1 = (mean_squared_error(target_valid_1, predictions_1))**0.5
ave_1 = target_1.mean()

print('RMSE модели =', rmse_1)
print('Средний запас сырья =', ave_1)

RMSE модели = 0.8902801001028828
Средний запас сырья = 68.82500000000002


Обучим модель для второго месторождения data_1

In [11]:
features_2 = data_2.drop(['id', 'product'], axis=1)
target_2 = data_2['product']
features_train_2, features_valid_2, target_train_2, target_valid_2 = (train_test_split(features_2, 
                                                                               target_2, 
                                                                               test_size = 0.25, 
                                                                               random_state=42))

In [12]:
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predictions_2 = model_2.predict(features_valid_2)
rmse_2 = (mean_squared_error(target_valid_2, predictions_2))**0.5
ave_2 = target_2.mean()

print('RMSE модели =', rmse_2)
print('Средний запас сырья =', ave_2)

RMSE модели = 40.145872311342174
Средний запас сырья = 95.00000000000004


## Вывод:

Постороив модель регрессии для трех месторождений мы видим следующее:
- В первом и третьем месторождения средний запас сырья выше, чем во втором.
- Во втором месторождении модель предсказания работает лучше.

# 3. Подготовка к расчёту прибыли

Запишем ключевые значения для расчетов

In [13]:
RESEARCH_POINT = 200 # Количество точек при разведке
BUDGET = 10000000000 # Бюджет на разработку
DRILLING_COST = 50000000 # Стоимость бурения одной скважины
BARREL_COST = 4500 # Прибыль с одного барреля.
BILLION = 1000000000

Посчитаем минимальный средний объём сырья в месторождении, достаточный для его разработки.

In [14]:
print('Минимальный средний объём сырья равен', DRILLING_COST // BARREL_COST, 'баррелям')

Минимальный средний объём сырья равен 11111 баррелям


### Вывод.
Учитывая что, минимальный объем сырья, достаточный для разработки, равен примерно 11 тыс. баррелей, все три региона подходят для разработки. Так как минимальный средний объем в регионе равен 68.8 тыс. баррелей

#### Напишем функцию расчета прибыли

In [15]:
def profit(target, predictions):
    pred_sorted = pd.Series(predictions).sort_values(ascending=False)
    selected = target[pred_sorted.index][:RESEARCH_POINT]
    revenue = BARREL_COST * (selected.sum() * 1000)
    return (revenue - BUDGET) / BILLION

# 4. Расчёт прибыли и рисков 

Создадим функцию Bootstrap, которая на вход получает целевой признак "target_valid" и предсказание модели "predictions"
А на выходе выдает список значений прибыли "values" 

In [16]:
state = RandomState(42)

def bootstrap(target_valid, predictions):
    values = []
    for i in range(1000):
        target_subsample = target_valid.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
        probs_subsample = pd.Series(predictions)[target_subsample.index]
        values.append(profit(target_subsample, probs_subsample))
    values = pd.Series(values)
    return values


In [17]:
state = RandomState(42)

def bootstrap(target_valid, predictions):
    values = []
    for i in range(1000):
        target_subsample = target_valid.reset_index(drop=True).sample(n=500, replace=True, random_state=state)
        probs_subsample = pd.Series(predictions)[target_subsample.index]
        values.append(profit(target_subsample, probs_subsample))
    values = pd.Series(values)
    confidence_interval = st.t.interval(0.95, len(values)-1, loc=values.mean(), scale=values.sem())
    risk = len(values[values < 0]) / len(values)
    
    print('Средняя прибыль в регионе = {:.2f} млрд'. format(values.mean()))
    print('1%-квантиль: {:.2f} млрд'. format(values.quantile(0.01)))
    print('95%-й доверительный интервал находится в промежутке между {:.2f} млрд и {:.2f} млрд'.format(confidence_interval[0], 
                                                                                                       confidence_interval [1]))
    print('Риск убытков равен {:.0%}'.format(risk))
    

##### Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков для первого региона.  

In [18]:
bootstrap(target_valid_0, predictions_0)

Средняя прибыль в регионе = 94.28 млрд
1%-квантиль: 87.62 млрд
95%-й доверительный интервал находится в промежутке между 94.11 млрд и 94.45 млрд
Риск убытков равен 0%


##### Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков для второго региона.

In [19]:
bootstrap(target_valid_1, predictions_1)

Средняя прибыль в регионе = 95.12 млрд
1%-квантиль: 90.39 млрд
95%-й доверительный интервал находится в промежутке между 94.98 млрд и 95.25 млрд
Риск убытков равен 0%


##### Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков для третьего региона.

In [20]:
bootstrap(target_valid_2, predictions_2)

Средняя прибыль в регионе = 94.09 млрд
1%-квантиль: 87.88 млрд
95%-й доверительный интервал находится в промежутке между 93.91 млрд и 94.26 млрд
Риск убытков равен 0%


## Выводы:

- Риски убытков во всех трех регионах равны нулю, так как значение 1%-ого квантиля везде намного выше, чем бюджет на разработку.
- Все три региона подходят для разработки, так как ни в одно регионе мы не понесем убытки.
- Самым перспективым является второй регион, потому что у него самая высокая прибыль (95 млрд), а модель предсказания запасов для этого региона имеет наименьшую погрешность.